In [1]:
import openai
import os

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [3]:
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


AttributeError: module 'openai' has no attribute 'OpenAI'

[](../images/03-Principle1.png)

## Prompting Principles
- **Principle 1: Write clear and specific instructions** (not short)
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [4]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To achieve the desired output from a model, provide clear and specific instructions, which may be longer for added clarity and context, to minimize irrelevant or incorrect responses.


![](../images/04-avoid-prompt-injection.png)

#### Tactic 2: Ask for a structured output
- JSON, HTML

In [5]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": "1",
    "title": "Whispers of the Ancient Forest",
    "author": "Evelyn Nightingale",
    "genre": "Fantasy"
  },
  {
    "book_id": "2",
    "title": "The Clockmaker's Paradox",
    "author": "Arthur C. Blackwood",
    "genre": "Science Fiction"
  },
  {
    "book_id": "3",
    "title": "Crimson Tides: A Detective's Memoir",
    "author": "Samantha Rhodes",
    "genre": "Mystery"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [6]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste, if desired.
Step 7 - Enjoy your delicious cup of tea.


In [7]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [8]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The mightiest oak in the forest was once a tiny acorn that stood its ground; the most enduring mountains have withstood the test of time and elements; the most traveled paths were built by the persistent steps of the determined traveler.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [9]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Summary:
Jack and Jill, siblings from a village, have an accident while fetching water but return home with their adventurous spirits intact.

French Summary:
Jack et Jill, frère et sœur d'un village, ont un accident en cherchant de l'eau mais rentrent chez eux avec leur esprit d'aventure intact.

Names in the French summary:
- Jack
- Jill

JSON Object:
```json
{
  "french_summary": "Jack et Jill, frère et sœur d'un village, ont un accident en cherchant de l'eau mais rentrent chez eux avec leur esprit d'aventure intact.",
  "num_names": 2
}
```


#### Ask for output in a specified format

In [10]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings from a village, had an accident while fetching water but remained adventurous.

Translation: Jack et Jill, frère et sœur d'un village, ont eu un accident en cherchant de l'eau mais sont restés aventureux.

Names: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill, frère et sœur d'un village, ont eu un accident en cherchant de l'eau mais sont restés aventureux.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [11]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is almost correct, but there is a small mistake in the maintenance cost calculation. The maintenance cost should be $100,000 plus $10 per square foot, not $100 per square foot.

Here's the corrected calculation:

Let x be the size of the installation in square feet.

Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + ($10 * x)

Total cost: Land cost + Solar panel cost + Maintenance cost
Total cost: (100x) + (250x) + (100,000 + 10x)
Total cost: 100x + 250x + 10x + 100,000
Total cost: (100 + 250 + 10)x + 100,000
Total cost: 360x + 100,000

So the correct total cost for the first year of operations as a function of the number of square feet (x) is 360x + 100,000.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [12]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x (The student incorrectly wrote 100x instead of 10x)
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [13]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

As of my last update in early 2023, the AeroGlide UltraSlim Smart Toothbrush by Boie is not a widely recognized product in the market. Boie is known for its eco-friendly oral care products, including toothbrushes with replaceable heads and biodegradable materials. However, there is no specific information available about an "AeroGlide UltraSlim Smart Toothbrush" by Boie.

It's possible that this product could be a new release or an upcoming product that has not yet been widely publicized or reviewed. If Boie has indeed launched a smart toothbrush under this name, it would likely feature some of the following characteristics based on trends in smart toothbrush technology:

1. Connectivity: The toothbrush might connect to a smartphone app via Bluetooth to provide feedback on brushing habits, such as duration, pressure, and coverage.

2. Brushing Modes: Multiple brushing modes for different needs, such as daily cleaning, gum care, whitening, or sensitive teeth.

3. Timer: A built-in timer

## Try experimenting on your own!

In [14]:
prompt = f"""

"""
response = get_completion(prompt)
print(response)

Hello! It seems like your message may have been cut off or not included. How can I assist you today? If you have a question or need information on a specific topic, feel free to ask!


![](../images/05-model-limitations.png)